In [2]:
import torch
from PIL import Image
from transformers import AutoProcessor, CLIPModel, AutoImageProcessor, AutoModel
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from scipy.spatial.distance import directed_hausdorff
import cv2 as cv
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity


device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

# Load CLIP model and processor
processor_clip = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
model_clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

# Load DINOv2 model and processor
processor_dino = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model_dino = AutoModel.from_pretrained('facebook/dinov2-base').to(device)

# Load VGG16 model
vgg16 = VGG16(weights='imagenet', include_top=False, pooling='max', input_shape=(224, 224, 3))

# Freezing the layers
for model_layer in vgg16.layers:
    model_layer.trainable = False

# Function to extract CLIP features
def extract_features_clip(image):
    with torch.no_grad():
        inputs = processor_clip(images=image, return_tensors="pt").to(device)
        image_features = model_clip.get_image_features(**inputs)
        return image_features

# Function to extract DINOv2 features
def extract_features_dino(image):
    with torch.no_grad():
        inputs = processor_dino(images=image, return_tensors="pt").to(device)
        outputs = model_dino(**inputs)
        image_features = outputs.last_hidden_state
        return image_features.mean(dim=1)

# Function to calculate cosine similarity between two feature vectors
def cosine_similarity2(vector1, vector2):
    vector1 = vector1 / vector1.norm(dim=-1, keepdim=True)
    vector2 = vector2 / vector2.norm(dim=-1, keepdim=True)
    return torch.sum(vector1 * vector2, dim=-1).item()

# Function to compute SSIM between two images
def compute_ssim(image1, image2):
    image1_np = np.array(image1)
    image2_np = np.array(image2)
    return ssim(image1_np, image2_np, multichannel=True)

# Function to compute MSE between two images
def compute_mse(image1, image2):
    image1_np = np.array(image1).astype(np.float32)
    image2_np = np.array(image2).astype(np.float32)
    return mean_squared_error(image1_np.flatten(), image2_np.flatten())

# Retrieve all filenames from the two directories
def get_image_paths(directory):
    images = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('jpg'):
                images.append(os.path.join(root, file))
    images.sort()  # Ensure images are in the same order in both directories
    return images

# Resize images to the minimum dimensions between two images
def resize_to_minimum(img1, img2):
    min_width = min(img1.width, img2.width)
    min_height = min(img1.height, img2.height)
    img1_resized = img1.resize((min_width, min_height))
    img2_resized = img2.resize((min_width, min_height))
    return img1_resized, img2_resized

# Function to load and preprocess an image for VGG16
def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded)

# Function to get VGG16 embeddings
def get_image_embeddings(img_array):
    return vgg16.predict(img_array)

# Function to calculate similarity score using VGG16 embeddings
def get_similarity_score(first_image_path, second_image_path):
    first_image_array = load_and_preprocess_image(first_image_path)
    second_image_array = load_and_preprocess_image(second_image_path)
    first_image_embedding = get_image_embeddings(first_image_array)
    second_image_embedding = get_image_embeddings(second_image_array)
    return cosine_similarity(first_image_embedding, second_image_embedding)[0][0]

# Function to calculate normalized Hausdorff distance
def normalized_hausdorff_distance(img1, img2):
    edges1 = cv.Canny(img1, 100, 200)
    edges2 = cv.Canny(img2, 100, 200)
    points1 = np.column_stack(np.where(edges1 > 0))
    points2 = np.column_stack(np.where(edges2 > 0))
    d_forward = directed_hausdorff(points1, points2)[0]
    d_reverse = directed_hausdorff(points2, points1)[0]
    hausdorff_dist = max(d_forward, d_reverse)
    height, width = img1.shape
    diagonal = np.sqrt(width**2 + height**2)
    return hausdorff_dist / diagonal

# Function to calculate SSIM
def get_ssim_score(img1_path, img2_path, resize=True):
    img1 = cv.imread(img1_path)
    img2 = cv.imread(img2_path)

    if resize:
        img1 = cv.resize(img1, (32, 32), interpolation=cv.INTER_CUBIC)
        img2 = cv.resize(img2, (32, 32), interpolation=cv.INTER_CUBIC)
    if img1.shape != img2.shape:
        return -1
    win_size = min(img1.shape[0], img1.shape[1], 31)
    score, _ = ssim(img1, img2, win_size=win_size, channel_axis=-1, full=True)
    return score

for (exp, dir1, dir2) in [
    ('C', './images_dataset/', './sd-turbo/'), ]:
    # Paths to the two directories containing the images
    # exp = 'C'
    # dir1 = './survey_images/original/'
    # dir2 = './survey_images/compressed2/'

    # Get the sorted image paths for both directories
    images1 = [os.path.join(dir1, f) for f in sorted(os.listdir(dir1)) if f.lower().endswith(('.jpg', '.webp', '.jpeg', '.png', '.bmp', '.tiff', '.gif'))]
    images2 = [os.path.join(dir2, f) for f in sorted(os.listdir(dir2)) if f.lower().endswith(('.jpg', '.webp', '.jpeg', '.png', '.bmp', '.tiff', '.gif'))]

    # Ensure both directories have the same number of images
    assert len(images1) == len(images2), "The directories do not contain the same number of images."

    # Initialize a list to store results
    results = []

    # Iterate over the paired images and calculate similarity scores
    for img1_path, img2_path in tqdm(zip(images1, images2), total=len(images1), desc="Calculating similarities"):
        img1 = Image.open(img1_path).convert('RGB')
        img2 = Image.open(img2_path).convert('RGB')
        img1_resized, img2_resized = img1.resize((224, 224)), img2.resize((224, 224))
        img1_np, img2_np = np.array(img1_resized), np.array(img2_resized)

        # Extract features using CLIP
        clip_features1 = extract_features_clip(img1_resized).to(device)
        clip_features2 = extract_features_clip(img2_resized).to(device)
        clip_similarity = cosine_similarity2(clip_features1, clip_features2)
        
        # Extract features using DINOv2
        dino_features1 = extract_features_dino(img1_resized).to(device)
        dino_features2 = extract_features_dino(img2_resized).to(device)
        dino_similarity = cosine_similarity2(dino_features1, dino_features2)
        
        # Calculate similarity metrics
        vgg16_similarity = get_similarity_score(img1_path, img2_path)
        hausdorff_similarity = normalized_hausdorff_distance(cv.cvtColor(img1_np, cv.COLOR_RGB2GRAY), cv.cvtColor(img2_np, cv.COLOR_RGB2GRAY))
        ssim_similarity = get_ssim_score(img1_path, img2_path)
        true_ssim = get_ssim_score(img1_path, img2_path, resize=False)    
        
        # Store results
        results.append({
            "EXP": exp,
            "IMG": os.path.basename(img1_path),
            "IMG1": img1_path,
            "IMG2": img2_path,
            "VGG16": vgg16_similarity,
            "Structure": hausdorff_similarity,
            "Color": ssim_similarity,
            "CLIP": clip_similarity,
            "DINO": dino_similarity,
            "SSIM": true_ssim,
        })

    # Convert results to a DataFrame and save to CSV
    df_results = pd.DataFrame(results)
    file_name = "sd_turbo_scores.csv"
    if os.path.exists(file_name):
        df_results.to_csv(file_name, mode='a', header=False, index=False)
    else:
        df_results.to_csv(file_name, index=False)
    print("Similarity scores saved to image_similarity_scores.csv")


c:\Users\Danish\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Calculating similarities:   0%|          | 0/148 [00:00<?, ?it/s]c:\Users\Danish\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\clip\modeling_clip.py:491: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


1/1 [==============================] - 0s 72ms/step


Calculating similarities:   1%|          | 1/148 [00:00<02:09,  1.14it/s]

1/1 [==============================] - 0s 75ms/step


Calculating similarities:   1%|▏         | 2/148 [00:01<01:41,  1.43it/s]

...

1/1 [==============================] - 0s 68ms/step


Calculating similarities:  99%|█████████▉| 147/148 [01:24<00:00,  1.73it/s]

1/1 [==============================] - 0s 71ms/step


Calculating similarities: 100%|██████████| 148/148 [01:25<00:00,  1.74it/s]

Similarity scores saved to image_similarity_scores.csv


In [3]:
df = pd.read_csv('sd_turbo_scores.csv')

df['Structure'] = df['Structure'].clip(upper=1)
df['Structure'] = df['Structure'].clip(lower=0)
df['Color'] = df['Color'].clip(upper=1)
df['Color'] = df['Color'].clip(lower=0)
df['DINO'] = df['DINO'].clip(upper=1)
df['DINO'] = df['DINO'].clip(lower=0)

def contraharmonic_mean_three_numbers(num1, num2, num3):
    return (num1 ** 2 + num2 ** 2 + num3 ** 2) / (num1 + num2 + num3)

df['COMB'] = df.apply(lambda row: contraharmonic_mean_three_numbers(1 - row['Structure'], row['Color'], row['DINO']), axis=1)

df.to_csv('sd_turbo_scores_comb.csv', index=False)

In [51]:
import lpips

loss_func = lpips.LPIPS(net='alex') # best forward scores
# best forward scores means that it uses the network that was found to perform best in the validation study
# you can also use net='vgg' or 'squeeze' or 'squeeze_SP' for different networks

# Load images
# img0 = lpips.im2tensor(lpips.load_image(images1[0])) # RGB image from [-1,1]
# img1 = lpips.im2tensor(lpips.load_image(images2[0]))
def PSNR(original, compressed): 
    mse = np.mean((original - compressed) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse)) 
    return psnr 

# open sim_results2.csv and add new column with LPIPS score as well as PSNR
df = pd.read_csv("sim_results2.csv")
df['LPIPS'] = np.nan
df['PSNR'] = np.nan

for i in range(len(df)):
    img0 = lpips.load_image(df['IMG1'][i])
    img1 = lpips.load_image(df['IMG2'][i])
    
    # Check if image sizes match, if not, resize img1 to match img0
    if img0.shape != img1.shape:
        
        print("Resizng image from {} to {}".format(img1.shape, img0.shape))
        img1 = cv.resize(img1, (img0.shape[1], img0.shape[0]))
    
    # Convert images to tensors for LPIPS
    img0_tensor = lpips.im2tensor(img0)
    img1_tensor = lpips.im2tensor(img1)


    df['LPIPS'][i] = loss_func.forward(img0_tensor, img1_tensor).item()

    # Load images using OpenCV for PSNR calculation
    img0_cv = cv.imread(df['IMG1'][i])
    img1_cv = cv.imread(df['IMG2'][i])

    # Ensure images are the same size for PSNR calculation
    if img0_cv.shape != img1_cv.shape:
        img1_cv = cv.resize(img1_cv, (img0_cv.shape[1], img0_cv.shape[0]))
    
    # Compute PSNR score
    mse = compute_mse(img0_cv, img1_cv)
    if mse == 0:  # Prevent division by zero if images are identical
        df['PSNR'][i] = float('inf')
    else:
        df['PSNR'][i] = 10 * np.log10(255**2 / mse)

df.to_csv("sim_results3.csv", index=False)
print("LPIPS scores saved to image_similarity_scores.csv")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


c:\Users\Danish\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danish\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: c:\Users\Danish\AppData\Local\Programs\Python\Python310\lib\site-packages\lpips\weights\v0.1\alex.pth
Resizng image from (832, 1248, 3) to (836, 1254, 3)


C:\Users\Danish\AppData\Local\Temp\ipykernel_47384\92173202.py:39: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['LPIPS'][i] = loss_func.forward(img0_tensor, img1_tensor).item()
C:\Users\Danish\AppData\Local\Temp\ipykernel_47384\92173202.

Resizng image from (528, 800, 3) to (532, 800, 3)
Resizng image from (512, 512, 3) to (640, 640, 3)


...

Resizng image from (656, 928, 3) to (656, 929, 3)


C:\Users\Danish\AppData\Local\Temp\ipykernel_47384\92173202.py:39: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['LPIPS'][i] = loss_func.forward(img0_tensor, img1_tensor).item()
C:\Users\Danish\AppData\Local\Temp\ipykernel_47384\92173202.

Resizng image from (1528, 856, 3) to (1532, 862, 3)


C:\Users\Danish\AppData\Local\Temp\ipykernel_47384\92173202.py:39: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['LPIPS'][i] = loss_func.forward(img0_tensor, img1_tensor).item()
C:\Users\Danish\AppData\Local\Temp\ipykernel_47384\92173202.

NameError: name 'plt' is not defined

In [53]:

df.to_csv("sim_results3.csv", index=False)